# Projet

## Collecte de données

In [1]:
!pip install sparqlwrapper
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [27]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get casino image
query = """SELECT ?image {
  ?data wdt:P31 wd:Q133215;
        wdt:P18 ?image
}
LIMIT 128"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["image"]["value"],
        )
    )

In [28]:
import pandas as pd
dataframe = pd.DataFrame(array, columns=["image"])
dataframe = dataframe.astype(
    dtype={"image": "<U200"}
)
dataframe

,image
0,http://commons.wikimedia.org/wiki/Special:File...
1,http://commons.wikimedia.org/wiki/Special:File...
2,http://commons.wikimedia.org/wiki/Special:File...
3,http://commons.wikimedia.org/wiki/Special:File...
4,http://commons.wikimedia.org/wiki/Special:File...
...,...
123,http://commons.wikimedia.org/wiki/Special:File...
124,http://commons.wikimedia.org/wiki/Special:File...
125,http://commons.wikimedia.org/wiki/Special:File...
126,http://commons.wikimedia.org/wiki/Special:File...


In [29]:
import os
try:
    os.mkdir('images')
except:
    pass

Nous allons écrire une méthode pour extraire les exifs

Nous allons maintenant télécharger les images

In [30]:
import requests
import shutil
import os
import json

count = 0
def download_image(url):
    global metadata, count
    if count % 10 == 0:
        print("Téléchargé ", count, " images") 
    count += 1

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        nom = os.path.basename(url)[-30:]
        chemin = "images/"+nom
        with open(chemin, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

In [31]:
print("Début du téléchargement")
dataframe.image.apply(download_image)

Début du téléchargement
Téléchargé  0  images
Téléchargé  10  images
Téléchargé  20  images
Téléchargé  30  images
Téléchargé  40  images
Téléchargé  50  images
Téléchargé  60  images
Téléchargé  70  images
Téléchargé  80  images
Téléchargé  90  images
Téléchargé  100  images
Téléchargé  110  images
Téléchargé  120  images


0      200
1      200
2      200
3      200
4      200
      ... 
123    200
124    200
125    200
126    200
127    200
Name: image, Length: 128, dtype: int64

On enregistre les exifs

In [32]:
!pip install scikit-learn
import os, sys, json
from PIL import Image
from PIL.ExifTags import TAGS
import numpy
import math
from sklearn.cluster import KMeans


def extraire_donnees(image_chemin):
    imgfile = Image.open(image_chemin)
    try:
        exif_data = imgfile._getexif()
    except:
        print(f"les EXIFs n'ont pas pu être extraits depuis {image_chemin}")
        exif_data = {}
    image_metadata = dict()
    if exif_data:
        for tag, value in exif_data.items():
            if(isinstance(value, bytes)):
                continue
            if (not isinstance(value, int)) and (not isinstance(value, float)):
                value = f"{value}"
            if tag in TAGS:
                image_metadata[TAGS[tag]] = value
    return image_metadata
print("Fonction extraction OK")

Fonction extraction OK



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Extraction des couleurs

In [33]:
def extraire_couleurs(image_chemin):
    imgfile = Image.open(image_chemin)
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    print(image_chemin)
    clusters = KMeans(n_clusters=2, n_init=2)
    couleurs = []
    try :
        clusters.fit(numarray)
        for i in range(2):
            couleur = ("#%02x%02x%02x"
                % (
                    math.ceil(clusters.cluster_centers_[i][0]),
                    math.ceil(clusters.cluster_centers_[i][1]),
                    math.ceil(clusters.cluster_centers_[i][2]),
                ))
            couleurs.append(couleur)
    except:
        print(f"les couleurs n'ont pas pu être extraits depuis {image_chemin}")
        couleur = "#00000"
        for i in range(2):
            couleurs.append(couleur)
    print(couleurs)
    return couleurs

In [34]:
def ajout_tag(dat, filename, tags):
    dat[filename]['tags'] = tags
    return dat

def lire_entree(dat, filename):
    input_file = "user_input/"+filename+".txt"
    tagstr = ""
    if os.path.exists(input_file):
        with open(input_file, 'r') as fichier:
            tagstr = fichier.read()
    tags = [partie for partie in tagstr.split() if partie]
    return ajout_tag(dat, filename, tags)

Construction du JSON

In [35]:
json_dat = {}
for filename in os.listdir("images"):
    json_dat[filename] = {
        'exif': extraire_donnees("images/"+filename),
        'color': extraire_couleurs("images/"+filename)
    }
    json_dat = lire_entree(json_dat, filename)
print(json_dat)
json.dump(json_dat, open("img_data.json", "w"), indent=2)

images/%20Casino%2C%20Las%20Vagas.jpg
['#92a0b0', '#4a4747']
images/%20Casino%2C%20Scarborough.jpg
['#d8d8db', '#42403f']
images/%20Montbenon%2C%20Lausanne.jpg
['#1e1b19', '#939da4']
images/%20Neuenahr%20Kurhaus%2030.JPG
['#313b57', '#beb8b0']
images/%20Pablo%20Lytton%20Casino.jpg
['#d9e3d9', '#545352']
images/%20Savoie%20-Aix-les-Bains.JPG
['#455549', '#a2bed1']
images/%20Vegas%20December%202013.jpg
['#b9bec2', '#5f5448']
images/%20Vegas%20Wynn%20P4220706.jpg
['#4c5248', '#b0c4d6']
images/%C3%A1rkert%20kaszin%C3%B3.jpg
['#a9afb1', '#423f3e']
images/-%20en%20Cultuurpaleis%201.jpg
['#3f3e41', '#869aba']
images/-%20North%20-%202011-06-04.jpg
['#505551', '#c9d1e2']
images/02017%2003%20%28cropped%29.jpg
['#97acc5', '#444342']
images/0Casino%20main%20entrance.jpeg
['#86a7dc', '#5c5446']
images/0Santa%20Ana%20Pueblo%20NM.jpg
['#a8b6c7', '#624b3e']
images/2015%20img38%20Spa%20house.jpg
['#7b7b70', '#227ee6']
images/20expansion%20april%202008.jpg
['#58686f', '#dfe9ef']
images/20Grand%20River%2

Maintenant, on a les données à l'état brut. On va les transformer pour garder des données traitables pour l'analyse. Il faut qu'on convertisse chaque donnée en une valeur traitable pour faire des vecteurs. On commence par une fonction de transformation de la première couleur dominante en tag

In [11]:
!pip install colorsysx


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import colorsysx as colorsys

couleurs_tags_possibles = {
    'red': [255, 0, 0],
    'green': [0, 255, 0],
    'blue': [0, 0, 255],
    'yellow': [255, 255, 0],
    'cyan': [0, 255, 255],
    'magenta': [255, 0, 255],
    'black': [0, 0, 0],
    'white': [255, 255, 255]
}

tags_keys = couleurs_tags_possibles.keys()
def rgb_to_hls(color_rgb):
    return colorsys.rgb_to_hls(*[v/255 for v in color_rgb[:3]])

colors_lab = {k: rgb_to_hls(couleurs_tags_possibles[k]) for k in tags_keys}
color_voisins = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(np.array([colors_lab[k] for k in tags_keys]))

def tag_de_couleur(colorhex):
    # hex to rgb: #xxxxxx -> [ x, x, x ]
    hex = colorhex.lstrip('#')
    color = tuple(int(hex[i:i+2], 16) for i in (0, 2, 4))
    
    color_lab = rgb_to_hls(color)
    distances, indices = color_voisins.kneighbors([color_lab])
    nearest_color = list(tags_keys)[indices[0][0]]
    return nearest_color

In [50]:
"""
 single_picdata peut être un dictionnaire {exif, color, tags} ou {}
 La sortie doit être un tableau (ou vecteur) traitable pour l'analyse
"""
def transform_data(single_picdata):
    keys = single_picdata.keys()
    exifs = single_picdata["exif"] if "exif" in keys else {}
    exifkeys = exifs.keys()
    tags = single_picdata["tags"]
    return [
        exifs["Orientation"] if "Orientation" in exifs else -1,
        tag_de_couleur(single_picdata["color"][0]),
        "true" if "old" in tags else "false",
        "true" if "modern" in tags else "false",
        "true" if "architecture" in tags else "false",
        "true" if "classy" in tags else "false",
        "true" if "nature" in tags else "false"
    ]

On initialise ensuite un tableau avec nos données traitables pour 10 images aléatoires

In [117]:
from random import sample
import json, os

def read_pic_datas(picnames):
    with open('img_data.json', 'r') as f:
        pic_json_data = json.load(f)
    pic_json_keys = pic_json_data.keys()
    res_dat = []
    for picname in picnames:
        single_picdat = pic_json_data[picname] if picname in pic_json_keys else {}
        res_dat.append(transform_data(single_picdat))
    return res_dat

pics = sample(os.listdir("images"), 10)
picdata_matrix = read_pic_datas(pics)

On demande les préférences utilisateur sur ces 10 images

In [15]:
!pip install ipywidgets


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [118]:
import ipywidgets as widgets
from IPython.display import clear_output

pic_output = widgets.Output()

shown_image_idx = 0
likes = [None] * len(pics)

def show(nomfichier):
    global pic_output
    with open("images/"+nomfichier, 'rb') as f:
        shown_image = f.read()
    picwidget = widgets.Image(
        value=shown_image,
    )
    picwidget.layout.height= '150px'
    with pic_output:
        clear_output()
        display(picwidget)

def clear_if_maxed():
    global shown_image_idx
    if shown_image_idx >= len(pics):
        with pic_output:
            clear_output()
            print("Fin de la navigation")
def likeit(_b):
    global shown_image_idx
    clear_if_maxed()
    likes[shown_image_idx] = True
    shown_image_idx += 1
    clear_if_maxed()
    show(pics[shown_image_idx])

def hateit(_b):
    global shown_image_idx, liked
    print("Hate")
    clear_if_maxed()
    likes[shown_image_idx] = False
    shown_image_idx += 1
    clear_if_maxed()
    show(pics[shown_image_idx])

like_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Like',
    icon='heart'
)
like_button.style = {'button_color': '#ff4757'}
like_button.on_click(likeit)

hate_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Dislike',
    icon='thumbs-down'
)
hate_button.style = {'button_color': '#5352ed'}
hate_button.on_click(hateit)
buttons = widgets.HBox([like_button, hate_button])

display(buttons,pic_output)
show(pics[shown_image_idx])

Output()

In [119]:
print(likes)

[False, True, False, True, False, False, False, False, False, True]


In [19]:
! pip3 install graphviz pydotplus

     ---------------------------------------- 47.1/47.1 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 278.7/278.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24578 sha256=3fc81c7987b38174f1ba9325d92340bdc0a66ed0af55ec863db46a37bcff9be8
  Stored in directory: c:\users\leo-b\appdata\local\pip\cache\wheels\85\58\96\6cba53da2c6e8690dd0a33f67a4a0e2e88f70798e9d4955812
Successfully built pydotplus



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [133]:
from random import sample
import json, os


def read_pic_datas2(picnames):
    with open('img_data.json', 'r') as f:
        pic_json_data = json.load(f)
    pic_json_keys = pic_json_data.keys()
    res_dat = []
    for picname in picnames:
        single_picdat = pic_json_data[picname] if picname in pic_json_keys else {}
        res_dat.append(transform_data(single_picdat))
    return res_dat


pics = sample(os.listdir("images"), 128)
picdata_matrix2 = read_pic_datas(pics)
print(picdata_matrix2[5])

[1, 'white', 'false', 'false', 'false', 'false', 'false']


In [149]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

label_encoders = [LabelEncoder() for _ in range(len(picdata_matrix2[0]))]
encoded_data = []
for i, column in enumerate(zip(*picdata_matrix)):
    encoded_data.append(label_encoders[i].fit_transform(column))

X = list(zip(*encoded_data))  # Features
y = likes  # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the logistic regression classifier
classifier = svm.SVC()
classifier.fit(X_train, y_train)

# Predictions
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(picdata_matrix2)
# Sample prediction
for j in picdata_matrix2:
    encoded_item = [label_encoders[i].transform([val])[0] for i, val in enumerate(j)]
    print(encoded_item)
    prediction = classifier.predict([encoded_item])[0]
    print(f"Prediction for the sample item: {prediction}")

Accuracy: 0.5
[[1, 'black', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'true', 'false', 'false', 'true'], [-1, 'black', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'true', 'false', 'true', 'false'], [1, 'white', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'true', 'false', 'false'], [1, 'black', 'false', 'false', 'false', 'false', 'false'], [1, 'white', 'false', 'false', 'true', 'false', 'false'], [1, 'black', 'false', 'false', 'true', 'false', 'true'], [1, 'black', 'false', 'false', 'true', 'false', 'true'], [1, 'white', 'false', 'false', 'true', 'false', 'false'], [1, 'cyan', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'false', 'false', 'false'], [-1, 'black', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'true', 'false', 'fa

ValueError: y contains previously unseen labels: 'true'

In [125]:
print(picdata_matrix)

[[1, 'magenta', 'false', 'false', 'false', 'true', 'false'], [-1, 'blue', 'false', 'true', 'false', 'false', 'false'], [1, 'white', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'false', 'false', 'false'], [-1, 'black', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'false', 'false', 'false'], [-1, 'red', 'false', 'false', 'false', 'true', 'false'], [1, 'white', 'false', 'false', 'false', 'false', 'false'], [1, 'white', 'false', 'false', 'false', 'false', 'false'], [1, 'black', 'false', 'false', 'false', 'false', 'false']]
